In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.linear_model import LinearRegression

le = LabelEncoder()

In [2]:
database = pd.read_csv('COBRA-YTD2017.csv')

In [3]:
database.rename(columns={'UC2 Literal' : 'Crime', 'occur_time': 'time', 'MaxOfnum_victims': 'victims', 'Avg Day': 'day'}, inplace=True)
# database.head()

In [4]:
# NOTE : I removed rows where victims are only 1, it screws up the results.
# database.day.unique()
arr = [23, 24, 21, 19, 27, 17, 15]
database = database[database.victims != 23]
database = database[database.victims != 24]
database = database[database.victims != 21]
database = database[database.victims != 19]
database = database[database.victims != 27]
database = database[database.victims != 17]
database = database[database.victims != 15]
# database = database[database.Crime != 'LARCENY-FROM VEHICLE']
array = database.Crime.unique()

In [5]:
# NOTE: Clean up database and remove strings basically.

arr = []

# NOTE: Crime column.
array = le.fit((np.array(database.Crime.unique())).tolist())
database.Crime = le.transform((np.array(database.Crime)).tolist())

# NOTE: Day column
le.fit(np.array(database.day.unique()).tolist())
database.day = le.transform((np.array(database.day)).tolist())

# NOTE : Neighborhood
le.fit(np.array(database.neighborhood.unique()).tolist())
database.neighborhood = le.transform(np.array(database.neighborhood).tolist())

database.victims = database.victims.fillna(0)

# NOTE : Extract hour
database['hour'] = pd.to_datetime(database.time).dt.hour

# NOTE : Extract month
database['month'] = pd.to_datetime(database.occur_date).dt.month

# NOTE : Extract day
database['days'] = pd.to_datetime(database.occur_date).dt.day

# Take relevant data to different DataFrame
df = database[['Crime', 'day', 'days', 'hour', 'victims', 'neighborhood', 'x', 'y', 'month']]

# Make dummy variables
df = df.join(pd.get_dummies(df.hour, prefix="hour"))
df = df.join(pd.get_dummies(df.days, prefix="days"))
df = df.join(pd.get_dummies(df.month, prefix="month"))
df = df.join(pd.get_dummies(df.day, prefix="day"))

arr = (database.groupby(['neighborhood']))['victims'].unique()
count = (database.groupby(['neighborhood']))['victims'].nunique()

# Did the LinearRegression thingy
# for i in range(0, database.neighborhood.nunique()):
#     arr[i] = sum(arr[i])/ count[i]
#     df.loc[df.neighborhood == i, 'victims'] = arr[i]
    
# df.victims = df.victims.astype(int)

In [6]:
X_all = df.drop(['Crime'], 1)
y_all = df['Crime']

X_all = X_all.fillna(0)

In [7]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, random_state = 2)

c:\users\kshit\onedrive\documents\github\atlanta_crime\venv\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
import matplotlib.pyplot as plt

classif = OneVsRestClassifier(SVC(kernel = 'rbf'))
# classif.fit(X_train, y_train)

# classif = RandomForestClassifier()
# classif = DecisionTreeClassifier(max_depth = 3)
# classif = LinearRegression()
classif.fit(X_train, y_train)

y_pred = classif.predict(X_train)

# print(np.mean((y_pred - y_train) **2))

print(sum(y_train == y_pred) / float(len(y_pred)))
# print(precision_score(y_train, y_pred, average='micro'))
# print(recall_score(y_train, y_pred, average='weighted'))
# print(f1_score(y_train, y_pred, average='macro'))

y_pred = classif.predict(X_test)

# print(np.mean((y_pred - y_test) **2))
# print(y_pred)

print(sum(y_test == y_pred) / float(len(y_pred)))
precision, recall, _ = precision_recall_curve(y_test, classif.decision_function(X_test))
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('11-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))


# export_graphviz(classif, out_file='tree.dot')
# print(precision_score(y_test, y_pred, average='micro'))
# print(f1_score(y_test, y_pred, average='micro'))

0.6938795853269537
0.38621411483253587


ValueError: multiclass format is not supported